In [1]:
# Installation
!pip install pytorch-pretrained-bert pytorch-nlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 837.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 41.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.51
    Uninstalling botocore-1.34.51:
      Successfully uninstalled botocore-1.34.51
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.1 requires botocore<1.34.52,>=1.34.41, but you have botocore 1.29.165 which is incompatible.


In [2]:
# Library imports

from collections import defaultdict
import random
import json
import pandas as pd
import re
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import nltk
from sklearn.metrics import precision_recall_fscore_support

2024-03-22 15:28:58.302399: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-22 15:28:58.302500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-22 15:28:58.457956: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [12]:
# Paths of json

train_json_path = '/kaggle/input/project-files-climate/train.json'
external_json_path = '/kaggle/input/project-files-climate/external.json'
dev_json_path = '/kaggle/input/project-files-climate/dev.json'
test_json_path = '/kaggle/input/project-files-climate/test-unlabelled.json'

In [13]:
# Defining paths for various datasets and mappings used in a data transformation process, including training, development, and testing data, stored in both TSV and JSON formats

First = 2
Last = 8

# transform data
train_tsv_path = 'train28.tsv'
dev_tsv_path = 'dev28.tsv'
test_tsv_path = 'test28.tsv'
dev_map_path = 'dev28_map.json'
test_map_path = 'test28_map.json'

In [14]:
# The provided code performs data preprocessing tasks, including converting JSON-formatted text data into TSV format and splitting articles into paragraphs, preparing the data for further analysis or modeling

def JsonToTsv(target_file, *files):
    texts = []
    labels = []
    for file in files:

        df = pd.read_json(file)
        text = list(df.loc['text'].values)
        try:
            label = list(df.loc['label'].values)
        except:
            label = [0 for _ in range(len(text))]
        texts.extend(text)
        labels.extend(label)

    df = pd.DataFrame({
        'sentence':texts,
        'label':labels
    })

    df.to_csv(target_file, sep='\t', index=False, header=True)

import re
def tsvArticleToTsvPara(source, target):
    df = pd.read_csv(source, sep='\t')
    all_paras = []
    all_labels = []

    d = defaultdict(int)
    para_id = 0

    for i, (texts, label) in enumerate(df.values):

        paras = re.split('\n', texts)
        paras = [p for p in paras if len(p)>10]
        para_index = [i for i in range(len(paras))]


        former = para_index[:2]
        later = para_index[-8:]
        index = former + later
        for _ in range(len(index)):
          d[para_id] = i
          para_id+=1
        all_paras.extend([paras[i] for i in index])
        all_labels.extend([label for _ in index])


    df = pd.DataFrame({
        'sentence':all_paras,
        'label':all_labels
    })
    df.to_csv(target, sep='\t', index=False, header=True)
    return d


def generate_all_para(name):
    for n in ['train', 'dev', 'test']:
        if n == 'train':
            ofs = [train_json_path, external_json_path]
        elif n == 'test':
            ofs = [test_json_path,]
        else:
            ofs = [dev_json_path,]
        JsonToTsv(n+'.tsv', *ofs)
        d = tsvArticleToTsvPara(n+'.tsv', n+name+'.tsv')
        with open(n+name+'_map.json', 'w') as f:
            json.dump(d, f)

In [15]:
# Executes the data transformation process, generating paragraph-level TSV files and corresponding mapping JSON files for the specified range of datasets, typically denoted by the concatenation of the "First" and "Last" variables
generate_all_para(str(First)+str(Last))

In [16]:
# GPU Test

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU: {}'.format(device_name))

Found GPU: /device:GPU:0


In [17]:
# GPU Name

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

'Tesla T4'

In [18]:
# Reading the training data from a TSV file specified by `train_tsv_path` into a pandas DataFrame (`trn_df`). It expects the TSV file to have columns named 'sentence' and 'label'. Finally, it retrieves the shape of the DataFrame, indicating the number of rows and columns in the training data

trn_df = pd.read_csv(train_tsv_path, delimiter='\t', header=0, names=['sentence', 'label'])
trn_df.shape

(22767, 2)

In [19]:
# Defining a function to segment input texts into sentences using NLTK's sentence tokenizer and formats them for BERT model input, while downloading the necessary NLTK data for tokenization

nltk.download('punkt')
def get_segmented_data(texts):
    segmenter = nltk.data.load('tokenizers/punkt/english.pickle')
    segmented_texts = []
    for text in texts:
        text = text.split('\n')
        segmented_text = '[CLS] '
        for para in text:.
            for sentence in segmenter.tokenize(para):
                segmented_text += sentence + ' [SEP] '
        segmented_text = segmented_text[:-1]
        segmented_texts.append(segmented_text)
    return segmented_texts

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
# Segmenting sentences from the training data into BERT-compatible format using the defined function `get_segmented_data()` and extracts corresponding labels for classification from the training DataFrame
sentences = get_segmented_data(trn_df.sentence.values)
labels = [int(i) for i in trn_df.label.values]

In [21]:
# Initializing a BERT tokenizer from the 'bert-base-uncased' pre-trained model and tokenizes the segmented texts obtained from the training data. It then prints the tokenized form of the first sentence

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print (tokenized_texts[0])


100%|██████████| 231508/231508 [00:00<00:00, 2018372.79B/s]


Tokenize the first sentence:
['[CLS]', 'why', 'houston', 'flooding', 'isn', '‘', 't', 'a', 'sign', 'of', 'climate', 'change', '[SEP]']


In [22]:
# Converting the tokenized texts into token IDs using the BERT tokenizer, padding the sequences to a maximum length of 128 tokens, and creating attention masks to indicate the presence of actual tokens versus padded tokens for each sequence

MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# Create attention masks
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [23]:
# Spliting the input token IDs and attention masks into training and validation sets using a 90-10 split ratio while also splitting the corresponding labels accordingly, with a fixed random state for reproducibility

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

In [25]:
# Converting the training and validation input sequences, labels, and attention masks into PyTorch tensors for further processing with a neural network model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)


/tmp/ipykernel_34/2960893429.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_inputs = torch.tensor(train_inputs)
/tmp/ipykernel_34/2960893429.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  validation_inputs = torch.tensor(validation_inputs)
/tmp/ipykernel_34/2960893429.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
/tmp/ipykernel_34/2960893429.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detac

In [26]:
# Creating a PyTorch DataLoader for the training data, which is iterating over batches of input sequences, attention masks, and labels, using random sampling from the training dataset with a specified batch size of 32

batch_size = 32
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [27]:
# Creating a PyTorch DataLoader for the validation data, iterating over batches of input sequences, attention masks, and labels in sequential order, using a specified batch size of 32

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [28]:
# Initializing a BERT model for sequence classification with 2 labels (binary classification) based on the 'bert-base-uncased' pre-trained model, and moves the model parameters to the GPU for faster computation

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

100%|██████████| 407873900/407873900 [00:09<00:00, 41075219.85B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=Fa

In [29]:
# Setting up the parameters and their respective decay rates for the optimizer. Parameters with names containing 'bias', 'gamma', or 'beta' are assigned a weight decay rate of 0.01, while others are assigned a weight decay rate of 0.0

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [30]:
# Initializing the optimizer using the BertAdam optimizer with the specified grouped parameters, learning rate of 2e-5, and warmup proportion of 0.1
optimizer = BertAdam(optimizer_grouped_parameters, lr=2e-5, warmup=.1)


In [31]:
# Calculating the accuracy of predictions compared to the actual labels by flattening the predicted values and labels, and then computing the proportion of correctly predicted labels

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [32]:
# Training the BERT model for the specified number of epochs, iterating through each epoch and performing training and validation steps. During training, computing the loss, performs backpropagation, and updating the model parameters. During validation, evaluating the model's accuracy on the validation dataset

t = []
train_loss_set = []
epochs = 4

for _ in trange(epochs, desc="Epoch"):

    # Training

    model.train()

    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()

        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    print("Train loss: {}".format(tr_loss/nb_tr_steps))


    # Validation

    model.eval()

    # Tracking variables
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/pytorch_pretrained_bert/optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/python_arg_parser.cpp:1519.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Train loss: 0.23328859803910468


Epoch:  25%|██▌       | 1/4 [08:39<25:58, 519.59s/it]

Validation Accuracy: 0.94140625
Train loss: 0.07159126466852082


Epoch:  50%|█████     | 2/4 [17:19<17:19, 519.66s/it]

Validation Accuracy: 0.9557291666666666
Train loss: 0.024666933524020978


Epoch:  75%|███████▌  | 3/4 [25:59<08:39, 519.79s/it]

Validation Accuracy: 0.94140625
Train loss: 0.0102150945336652


Epoch: 100%|██████████| 4/4 [34:39<00:00, 519.88s/it]

Validation Accuracy: 0.95703125


In [33]:
# predict(tsv_file_path, model) reading a TSV file containing sentences and their labels, tokenizes the sentences, prepares them for prediction, and then making predictions using the provided BERT model and returning the predicted labels

def predict(tsv_file_path, model):
    df = pd.read_csv(tsv_file_path, delimiter='\t', header=0, names=['sentence', 'label'])

    sentences = df.sentence.values
    sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
    labels = [int(i) for i in df.label.values]

    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    MAX_LEN = 128
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    prediction_inputs = torch.tensor(input_ids)
    prediction_masks = torch.tensor(attention_masks)
    prediction_labels = torch.tensor(labels)

    batch_size = 32

    prediction_data = TensorDataset(prediction_inputs, prediction_masks, prediction_labels)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)


    model.eval()
    predictions , true_labels = [], []

    # Predict
    for batch in prediction_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        predictions.append(logits)
        true_labels.append(label_ids)

    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    return flat_predictions

In [34]:
# Using provided BERT model to make predictions on the development dataset (`dev_tsv_path`), returning the predicted labels

dev_flat_predictions = predict(dev_tsv_path, model)


In [35]:
# `read_data_from_json(file_path)` readin labels from a JSON file located at `file_path` and returning them as a list of integers, then reading the labels for the development dataset from a JSON file and loads the corresponding mapping JSON file

def read_data_from_json(file_path):
    df = pd.read_json(file_path)
    labels = [int(i) for i in df.loc['label'].values]
    return labels
dev_labels = read_data_from_json(dev_json_path)

with open(dev_map_path,'r') as f:
    map = json.load(f)

In [36]:
# Adjusts the predictions made on the development dataset (`dev_flat_predictions`) based on a mapping (`map`) and a threshold rule. It counts the number of positive predictions for each original instance, updates them based on the threshold (if more than 4), and then prints the count of positive predictions

new_predictions = [0] * len(dev_labels)
for i, result in enumerate(dev_flat_predictions):
    if result == 1:
        new_predictions[map[str(i)]] += 1

for i in range(len(new_predictions)):
    if new_predictions[i] >4:
        new_predictions[i] = 1
    else:
        new_predictions[i]=0

print(len([i for i in new_predictions if i==1]))

55


In [37]:
# Calculating the precision, recall, and F1-score metrics using the `precision_recall_fscore_support` function from scikit-learn, comparing the predicted labels (`new_predictions`) with the actual labels (`dev_labels`) for the positive class (pos_label=1). Then, it prints out the precision, recall, and F1-score values

p, r, f, _ = precision_recall_fscore_support(dev_labels, new_predictions, pos_label=1, average="binary")
print('precision:',p)
print('recall:',r)
print('f_score:',f)

precision: 0.8363636363636363
recall: 0.92
f_score: 0.8761904761904761


In [38]:
# Using the provided BERT model to make predictions on the test dataset (`test_tsv_path`), returning the predicted labels

test_flat_predictions = predict(test_tsv_path, model)


In [39]:
# Reading the mapping JSON file for the test dataset from the specified path and loads it into a dictionary named `test_map`

with open(test_map_path,'r') as f:
    test_map = json.load(f)


In [40]:
# Adjusting the predictions made on the test dataset (`test_flat_predictions`) based on a mapping (`test_map`) and a threshold rule. Counting the number of positive predictions for each original instance, updating them based on the threshold (if more than 4), and then printing the count of positive predictions

new_predictions = [0] * (max(test_map.values())+1)
for i, result in enumerate(test_flat_predictions):
    if result == 1:
        new_predictions[test_map[str(i)]] += 1

for i in range(len(new_predictions)):
    if new_predictions[i] >4:
        new_predictions[i] = 1
    else: new_predictions[i]=0

print(len([i for i in new_predictions if i>0]))

148


In [41]:
# Constructing a dictionary `test_dict` where each entry corresponds to a test instance. The predicted label for each instance is set to 1 if the prediction probability is greater than or equal to 0.5, otherwise, it's set to 0

test_dict = {}
for i in range(len(new_predictions)):
    label = 1 if new_predictions[i]>= 0.5 else 0
    test_dict['test-'+str(i)] = {'label':label}

In [42]:
# Defining a function `save_to_zip(pred_dict, zip_name)` that saves a dictionary `pred_dict` to a JSON file named 'test-output.json', and then creating a zip file with the given name (`zip_name`) containing the JSON file. The name of the zip file is based on the current date and time, truncated to the hour. Additionally, returning the current date and time truncated to the hour

import zipfile
import datetime
def save_to_zip(pred_dict, zip_name):
    with open('test-output.json', 'w') as f:
        json.dump(pred_dict, f)
    z = zipfile.ZipFile(zip_name+'.zip', 'w', zipfile.ZIP_STORED)
    z.write('test-output.json', 'test-output.json')
    z.close()
save_to_zip(test_dict, str(datetime.datetime.now())[:13])
str(datetime.datetime.now())[:13]

'2024-03-22 16'